In [1]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), ".."))


In [2]:
from notebooks.helpers import PreprocessingDataframe,EmbeddingModel

/Users/marcofura/anaconda3/envs/deep_learning_avanzado/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
pathname = '../datasets/MELD.Raw/dev_sent_emo.csv'
pathname_train = '../datasets/MELD.Raw/train/train_sent_emo.csv'
df = pd.read_csv(pathname)

In [4]:
embedding_model = EmbeddingModel(model_name='hubert')

In [5]:
preprocessing = PreprocessingDataframe(df,folder_audio_path='../datasets/MELD.Raw/dev_splits_complete')

In [6]:
preprocessing.set_path_wavfiles()

In [20]:
preprocessing.drop_na()

In [8]:
row_data = preprocessing.df.iloc[0]

In [9]:
row_data['Emotion']

'sadness'

In [10]:
embedding_audio=embedding_model.embed_audio(row_data['wav_file'])

Sample rate: 48000
Waveform shape before: torch.Size([6, 134144])
Bundle sample rate: 16000
Waveform shape: torch.Size([1, 44715])


In [11]:
embedding_text=embedding_model.embed_text(row_data['Emotion'])

Input type: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Inputs Shape: torch.Size([1, 3])


In [12]:
preprocessing.add_clap_embeddings(embedding_model)

Generating audio embeddings for  ../datasets/MELD.Raw/dev_splits_complete/dia0_utt0.wav
Sample rate: 48000
Waveform shape before: torch.Size([6, 134144])
Bundle sample rate: 16000
Waveform shape: torch.Size([1, 44715])
before saving in DF <class 'torch.Tensor'>
Generating text embeddings for  Oh my God, hes lost it. Hes totally lost it.
Input type: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Inputs Shape: torch.Size([1, 3])
before saving in DF  <class 'torch.Tensor'>
Generating audio embeddings for  ../datasets/MELD.Raw/dev_splits_complete/dia0_utt1.wav
Sample rate: 48000
Waveform shape before: torch.Size([6, 64512])
Bundle sample rate: 16000
Waveform shape: torch.Size([1, 21504])
before saving in DF <class 'torch.Tensor'>
Generating text embeddings for  What?
Input type: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Inputs Shape: torch.Size([1, 3])
before saving in DF  <class 'torch.Tensor'>
Generating audio embeddings for  ../datasets/MELD.Raw/dev_sp

In [13]:
preprocessing.df.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,wav_file,audio_embeddings,text_embeddings
0,1.0,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0.0,0.0,4.0,7.0,"00:20:57,256","00:21:00,049",../datasets/MELD.Raw/dev_splits_complete/dia0_...,"[[[tensor(-0.0102), tensor(-0.1646), tensor(0....","[[[tensor(-0.1934), tensor(0.2096), tensor(-0...."
1,2.0,What?,Monica,surprise,negative,0.0,1.0,4.0,7.0,"00:21:01,927","00:21:03,261",../datasets/MELD.Raw/dev_splits_complete/dia0_...,"[[[tensor(-0.0502), tensor(0.1232), tensor(0.2...","[[[tensor(-0.1820), tensor(0.1238), tensor(-0...."
2,3.0,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1.0,0.0,4.0,4.0,"00:12:24,660","00:12:30,915",../datasets/MELD.Raw/dev_splits_complete/dia1_...,"[[[tensor(-0.1483), tensor(-0.0067), tensor(0....","[[[tensor(-0.3377), tensor(0.2824), tensor(-0...."
3,4.0,Youre a genius!,Chandler,joy,positive,1.0,1.0,4.0,4.0,"00:12:32,334","00:12:33,960",../datasets/MELD.Raw/dev_splits_complete/dia1_...,"[[[tensor(0.2420), tensor(0.2939), tensor(0.11...","[[[tensor(-0.1841), tensor(0.1157), tensor(0.0..."
4,5.0,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1.0,2.0,4.0,4.0,"00:12:34,211","00:12:37,505",../datasets/MELD.Raw/dev_splits_complete/dia1_...,"[[[tensor(0.0073), tensor(-0.4655), tensor(-0....","[[[tensor(-0.1934), tensor(0.2096), tensor(-0...."


In [39]:
row = preprocessing.df.iloc[0]

In [40]:
row['audio_embeddings'].shape

torch.Size([1, 139, 768])

In [21]:
preprocessing.df['audio_embedding_mean'] = preprocessing.df['audio_embeddings'].apply(lambda x: x.mean(dim=1))
preprocessing.df['text_embedding_mean'] = preprocessing.df['text_embeddings'].apply(lambda x: x.mean(dim=1))

In [19]:
preprocessing.save_embeddings('df_embeddings_clap.pkl')

In [6]:
preprocessing.load_embeddings('df_embeddings_clap.pkl')

In [7]:
type(preprocessing.df['audio_embedding_mean'])

pandas.core.series.Series

In [38]:
import torch

# Promediar embeddings usando PyTorch
preprocessing.df['audio_embedding_mean'] = preprocessing.df['audio_embedding_mean'].apply(lambda x: torch.mean(x, dim=0).tolist())
preprocessing.df['text_embedding_mean'] = preprocessing.df['text_embedding_mean'].apply(lambda x: torch.mean(x, dim=0).tolist())

In [7]:
from notebooks.helpers.model_clap import EnhancedCLAPModel

In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(preprocessing.df, test_size=0.2, random_state=42)

In [9]:
from torch.utils.data import Dataset
import torch
# Dataset personalizado
class AudioTextDataset(Dataset):
    def __init__(self, dataframe):
        self.audio_embeddings = torch.tensor(dataframe['audio_embedding_mean'], dtype=torch.float32)
        self.text_embeddings = torch.tensor(dataframe['text_embedding_mean'], dtype=torch.float32)
    
    def __len__(self):
        return len(self.audio_embeddings)
    
    def __getitem__(self, idx):
        return self.audio_embeddings[idx], self.text_embeddings[idx]

In [11]:
import pandas as pd
import numpy as np

# Asegurarse de resetear el índice
val_df = val_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
# Garantizar que los embeddings sean listas y consistentes
val_df['audio_embedding_mean'] = val_df['audio_embedding_mean'].apply(lambda x: np.array(x, dtype=np.float32))
val_df['text_embedding_mean'] = val_df['text_embedding_mean'].apply(lambda x: np.array(x, dtype=np.float32))
train_df['audio_embedding_mean'] = train_df['audio_embedding_mean'].apply(lambda x: np.array(x, dtype=np.float32))
train_df['text_embedding_mean'] = train_df['text_embedding_mean'].apply(lambda x: np.array(x, dtype=np.float32))

# Verificar la consistencia de las columnas
print(val_df['audio_embedding_mean'].apply(type).value_counts())
print(val_df['text_embedding_mean'].apply(type).value_counts())
print(train_df['audio_embedding_mean'].apply(type).value_counts())
print(train_df['text_embedding_mean'].apply(type).value_counts())

<class 'numpy.ndarray'>    222
Name: audio_embedding_mean, dtype: int64
<class 'numpy.ndarray'>    222
Name: text_embedding_mean, dtype: int64
<class 'numpy.ndarray'>    885
Name: audio_embedding_mean, dtype: int64
<class 'numpy.ndarray'>    885
Name: text_embedding_mean, dtype: int64


In [12]:
from torch.utils.data import DataLoader
train_dataset = AudioTextDataset(train_df)
val_dataset = AudioTextDataset(val_df)

/var/folders/xw/ksr09pc96kqcwxk95_zhnv080000gn/T/ipykernel_4871/2578943347.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  self.audio_embeddings = torch.tensor(dataframe['audio_embedding_mean'], dtype=torch.float32)


In [13]:
def evaluate(model, val_loader, criterion, device):
    model.eval()  # Modo de evaluación
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():  # Evitar el cálculo de gradientes
        for audio_embeds, text_embeds in val_loader:
            audio_embeds, text_embeds = audio_embeds.to(device), text_embeds.to(device)
            
            # Forward pass
            similarity_matrix = model(audio_embeds, text_embeds)
            
            # Crear etiquetas (pares correctos en la diagonal)
            targets = torch.arange(similarity_matrix.size(0)).to(device)
            
            # Calcular pérdida
            loss = criterion(similarity_matrix, targets)
            total_loss += loss.item()
            
            # Calcular accuracy
            preds = torch.argmax(similarity_matrix, dim=1)
            total_correct += (preds == targets).sum().item()
            total_samples += targets.size(0)
    
    avg_loss = total_loss / len(val_loader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

In [14]:
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = 768
projection_dim = 512
batch_size = 32
epochs = 50


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Inicialización del modelo, optimizador y función de pérdida
model = EnhancedCLAPModel(input_dim=input_dim, projection_dim=projection_dim).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Entrenamiento
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for audio_embeds, text_embeds in train_dataloader:
        audio_embeds, text_embeds = audio_embeds.to(device), text_embeds.to(device)
        
        # Forward pass
        similarity_matrix = model(audio_embeds, text_embeds)
        
        # Crear etiquetas (pares correctos en la diagonal)
        targets = torch.arange(similarity_matrix.size(0)).to(device)
        
        # Calcular la pérdida
        loss = criterion(similarity_matrix, targets)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        train_accuracy = (torch.argmax(similarity_matrix, dim=1) == targets).float().mean()
        
    val_loss, val_accuracy = evaluate(model, val_dataloader, criterion, device)
    
    print(f"Epoch [{epoch+1}/{epochs}], "
          f"Train Loss: {total_loss/len(val_dataloader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


AssertionError: For unbatched (2-D) `query`, expected `key` and `value` to be 2-D but found 3-D and 3-D tensors respectively

In [24]:
print(f' Capas del modelo: {model}')
print(f' Parámetros del modelo: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')


 Capas del modelo: CLAPModel(
  (audio_projection): Linear(in_features=768, out_features=512, bias=True)
  (text_projection): Linear(in_features=768, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
 Parámetros del modelo: 787456
